In [1]:
import json
import pandas as pd

In [2]:
with open('./raw.json','r') as f:
    d = json.load(f)

In [3]:
d.keys()

dict_keys(['mapping', 'identifiers', 'entities', 'oldEntities', 'data'])

In [4]:
d['mapping']

['scenario',
 'country',
 'sector',
 'year',
 'indirect',
 'direct',
 'stock',
 'sales',
 'value',
 'consumption']

In [5]:
d['identifiers']

['scenario', 'country', 'sector', 'year']

In [6]:
d['entities']

['country', 'scenario', 'sector', 'subsector', 'region', 'regionSelect']

In [7]:
d['oldEntities']

{'formats': {'indirect': {'unit': 't', 'formatPrefix': 3, 'factor': 1000000.0},
  'direct': {'unit': 't', 'formatPrefix': 3, 'factor': 1000000.0},
  'total': {'unit': 't', 'format': '.3r'},
  'value': {'format': '.4r$'}},
 'labels': {'indirect': 'Indirect emissions',
  'direct': 'Direct emissions',
  'total': 'Total emissions',
  'erp': 'Emission reduction potential',
  'stock': 'Appliances in use',
  'sales': 'Unit sales',
  'value': 'Value',
  'allSectors': 'cooling sector'},
 'scenario': [{'id': 1, 'title': 'BAU', 'description': 'Business as Usual'},
  {'id': 2,
   'title': 'MIT',
   'description': 'Steht für Mitigation, d.h. dies sind die Emissionen im reduzierte Szenario'}],
 'sector': [{'id': 1, 'title': 'Unitary air conditioning'},
  {'id': 2, 'title': 'Chiller'},
  {'id': 3, 'title': 'Mobile AC'},
  {'id': 4, 'title': 'Domestic refrigeration'},
  {'id': 5, 'title': 'Commercial Refrigeration'},
  {'id': 6, 'title': 'Industrial Refrigeration'},
  {'id': 7, 'title': 'Transport Ref

In [8]:
d['oldEntities'].keys()

dict_keys(['formats', 'labels', 'scenario', 'sector', 'subsector', 'regionSelect', 'region', 'country'])

In [9]:
scenario_df = pd.DataFrame(d['oldEntities']['scenario']).set_index('id')
scenario_df

,title,description
id,,
1,BAU,Business as Usual
2,MIT,"Steht für Mitigation, d.h. dies sind die Emiss..."


In [10]:
sector_df = pd.DataFrame(d['oldEntities']['sector']).set_index('id')
sector_df

,title
id,
1,Unitary air conditioning
2,Chiller
3,Mobile AC
4,Domestic refrigeration
5,Commercial Refrigeration
6,Industrial Refrigeration
7,Transport Refrigeration


In [11]:
countries = pd.DataFrame(d['oldEntities']['country']).set_index('id')
countries.head()

,iso,name,temp,region
id,,,,
1,4.0,Afghanistan,40.0,2.0
2,8.0,Albania,32.0,6.0
3,12.0,Algeria,40.0,3.0
4,16.0,American Samoa,NaN,7.0
5,20.0,Andorra,NaN,NaN


In [12]:
data = pd.DataFrame(d['data'], columns=d['mapping'])

In [13]:
data['year'] = data['year'] + 2000
data

,scenario,country,sector,year,indirect,direct,stock,sales,value,consumption
0,1,1,1,2000,0.060100,0.247000,154000.00,17400.000,28300000.0,415.0000
1,1,1,1,2002,0.066800,0.275000,172000.00,20800.000,33500000.0,461.0000
2,1,1,1,2004,0.074400,0.308000,193000.00,23500.000,37300000.0,513.0000
3,1,1,1,2006,0.081600,0.340000,215000.00,25100.000,38900000.0,563.0000
4,1,1,1,2008,0.088400,0.372000,238000.00,27100.000,41000000.0,610.0000
...,...,...,...,...,...,...,...,...,...,...
73939,2,235,7,2042,0.000011,0.000002,2.29,0.152,364.0,0.0133
73940,2,235,7,2044,0.000010,0.000002,2.28,0.151,363.0,0.0131
73941,2,235,7,2046,0.000010,0.000002,2.28,0.151,362.0,0.0129
73942,2,235,7,2048,0.000010,0.000001,2.28,0.150,361.0,0.0128


In [14]:
data.to_csv('data.csv')

# Explore the observed vs projected cutoff

In [15]:
value_cols = ['indirect','direct','stock','sales','value','consumption']
grouped = data.groupby(['country','year','sector'])[value_cols]

In [16]:
grouped.size().value_counts()

2    36972
dtype: int64

In [17]:
# this is for each value, the difference between scenario 1 and scenario 2
diffs = (grouped.max() - grouped.min())

In [18]:
diffs.reset_index().groupby('year')[value_cols].mean()

,indirect,direct,stock,sales,value,consumption
year,,,,,,
2000,0.000000,0.000000,0.0,0.0,0.0,0.000000
2002,0.000000,0.000118,0.0,0.0,0.0,0.000000
2004,0.000000,0.000247,0.0,0.0,0.0,0.000000
2006,0.000000,0.000498,0.0,0.0,0.0,0.000000
2008,0.000000,0.000784,0.0,0.0,0.0,0.000000
2010,0.000000,0.001842,0.0,0.0,0.0,0.000000
2012,0.000000,0.001180,0.0,0.0,0.0,0.000000
2014,0.000000,0.001302,0.0,0.0,0.0,0.000000
2016,0.000000,0.000000,0.0,0.0,0.0,0.000000


Okay, we see some slight differences in direct emissions before 2018, but for the most part they are very similar. We'll use scenario 1.

# Create the aggregated past data

In [19]:
cleaned = data[(data.scenario == 1) & (data.year < 2018)].groupby(['country','year'])[value_cols].sum()
cleaned = cleaned.reset_index()
cleaned['country_name'] = cleaned['country'].map(countries.name)

Given my understanding of the columns, summing them across sectors is appropriate. But without further documentation we don't know for sure.

In [20]:
cleaned

,country,year,indirect,direct,stock,sales,value,consumption,country_name
0,1,2000,0.274180,0.888390,589415.000,53338.0000,53504000.0,1454.3900,Afghanistan
1,1,2002,0.305640,0.983370,658272.000,78230.4000,67818000.0,1609.3200,Afghanistan
2,1,2004,0.342710,1.094010,769247.000,104317.8000,79843000.0,1798.1500,Afghanistan
3,1,2006,0.379710,1.200120,919177.000,131928.2000,88635000.0,1991.5200,Afghanistan
4,1,2008,0.416710,1.302770,1114024.000,164951.5000,99574000.0,2178.1000,Afghanistan
...,...,...,...,...,...,...,...,...,...
1867,235,2008,0.000084,0.000619,277.256,18.4284,23859.0,2.0637,Wallis and Futuna Islands
1868,235,2010,0.000083,0.000621,279.256,18.7107,24295.0,2.0585,Wallis and Futuna Islands
1869,235,2012,0.000109,0.000645,286.888,22.8296,29591.0,2.1061,Wallis and Futuna Islands
1870,235,2014,0.000137,0.000674,296.532,24.0946,31395.0,2.1691,Wallis and Futuna Islands


In [21]:
cleaned.to_csv('cleaned.csv')